In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.dates as mdates

In [33]:
#读取时间段的按月某国报道广告数目 1
result = pd.read_csv('report_result_2.csv')
result['monthdate'] = pd.to_numeric(result['monthdate'], downcast='integer')
result['monthdate'] = pd.to_datetime(result['monthdate'].astype('str'),format = '%Y%m')

#读取时间段的按月某国被报道广告数目 2
result_reported = pd.read_csv('reported_result_2.csv')
result_reported['monthdate'] = pd.to_numeric(result_reported['monthdate'], downcast='integer')
result_reported['monthdate'] = pd.to_datetime(result_reported['monthdate'].astype('str'),format = '%Y%m')

#读取时间段的按月报道广告总数目 3 
month_nums = pd.read_csv('month_nums.csv')
month_nums['monthdate'] = pd.to_numeric(month_nums['monthdate'], downcast='integer')
month_nums['monthdate'] = pd.to_datetime(month_nums['monthdate'].astype('str'),format = '%Y%m')

#读取时间段内按月报告本国的数目 4
report_own_country = pd.read_csv('report_own_country.csv')
report_own_country['monthdate'] = pd.to_numeric(report_own_country['monthdate'], downcast='integer')
report_own_country['monthdate'] = pd.to_datetime(report_own_country['monthdate'].astype('str'),format = '%Y%m')


In [34]:
result_proportion = pd.DataFrame(columns=['monthdate','country_code','proportion'])
row_num = 0
#计算国家A占所有非国家A新闻中的百分比（例如提及美国的非美国新闻占所有非美国新闻的百分比）
#(某国家被报道总数2 - 报道本国总数4)/(该月总报道书3 - 某国报道数1) ** 0.5
for country in result['country_code'].unique():
    for month in result['monthdate'].unique():
        if(result[(result['monthdate'] == month) & (result['country_code'] == country)].empty == 0):
            num1 = result[(result['monthdate'] == month) & (result['country_code'] == country)].iloc[0,2]
        else:
            num1 = 0
            
        num3 = month_nums[month_nums['monthdate'] == month].iloc[0,1]
        
        if(result_reported[(result_reported['monthdate'] == month) & (result_reported['report_code'] == country)].empty == 0):
            num2 = result_reported[(result_reported['monthdate'] == month) & (result_reported['report_code'] == country)].iloc[0,2]
        else:
            num2 = 0
        
        if(report_own_country[(report_own_country['monthdate'] == month) & (report_own_country['report_code'] == country)].empty == 0):
            num4 = report_own_country[(report_own_country['monthdate'] == month) & (report_own_country['report_code'] == country)].iloc[0,2]
        else:
            num4 = 0
        
        proportion = (num2 - num4)/(num3 - num1) 
        
        #添加一行
        result_proportion.loc[row_num] = [month,country,proportion]
        row_num += 1

In [35]:
ex_month_total_id = result_proportion

In [39]:
#保存结果
new_dataframe = ex_month_total_id['country_code'].unique()
new_dataframe.sort()
result_frame = ['month']
for i in new_dataframe:
    result_frame.append(i)
result_pd = pd.DataFrame(columns = result_frame)
#将国家转化为列输出，空值为0
export_data = ex_month_total_id[['monthdate','country_code','proportion']]
for month in export_data['monthdate'].unique():
    fill_country_list = [month]
    for country in new_dataframe:
        if(export_data[(export_data['monthdate'] == month) & (export_data['country_code'] == country)].empty == 0):
            fill_country_list.append(export_data[(export_data['monthdate'] == month) & (export_data['country_code'] == country)].iloc[0,2])
        else:
            fill_country_list.append(0)
    result_pd.loc[len(result_pd)] = fill_country_list

In [40]:
result_pd.to_csv('201502_gdelt_reportnum.csv',index = False)